In [691]:
import pandas as pd
import numpy as np

In [692]:
training_csv = pd.read_csv("SBIC.v2.agg.trn.csv", index_col=0)

In [693]:
training_csv

,post,targetMinority,targetCategory,targetStereotype,whoTarget,intentYN,sexYN,offensiveYN,dataSource,hasBiasedImplication
0,"\n\nBill Kristol and Ben Shaprio, two turds in...",[],[],[],0.0,0.886667,0.0,1.000000,Gab,1
1,\n\nRose\n🌹Taylor‏ @RealRoseTaylor 6h6 hours a...,[],[],[],NaN,0.220000,0.0,0.000000,Gab,1
2,\nCharlie Kirk‏\n\nJohnny Depp calls for death...,"[""liberals""]","[""social""]","[""miscontrue things"", ""take things in a weird ...",1.0,1.000000,0.0,1.000000,Gab,0
3,\nDavid Knight‏ \n\nNotice how quickly things ...,[],[],[],0.0,0.110000,0.0,0.166667,Gab,1
4,\nFinland fireball: Time-lapse video shows nig...,[],[],[],NaN,0.000000,0.0,0.000000,Gab,1
...,...,...,...,...,...,...,...,...,...,...
35498,👉 Illegally in the country after 5 deportation...,[],[],[],NaN,0.220000,0.0,0.000000,Gab,1
35500,💥Breaking💥\nJulian Assange is the gate keeper ...,[],[],[],0.0,0.000000,0.0,0.000000,Gab,1
35501,📖 2Kings 22:19 because your heart was peniten...,[],[],[],NaN,0.000000,0.0,0.000000,Gab,1
35502,🚨#FAKENEWSAWARDS🚨\n\n🚨 who is #1 fake news ?🚨\...,[],[],[],0.0,0.553333,0.0,0.500000,Gab,1


# Manipulating the CSV file so that we only use the columns specified by Maarten
- post [CLS] offensiveYN [GRP] group [STE] implications [END]

In [694]:
training_csv = training_csv.drop(["dataSource", "offensiveYN", "sexYN", "intentYN", "targetCategory"], axis=1, index=None)
training_csv.columns = ["post", "group", "implications", "whoTarget", "offensiveYN"]
training_csv

,post,group,implications,whoTarget,offensiveYN
0,"\n\nBill Kristol and Ben Shaprio, two turds in...",[],[],0.0,1
1,\n\nRose\n🌹Taylor‏ @RealRoseTaylor 6h6 hours a...,[],[],NaN,1
2,\nCharlie Kirk‏\n\nJohnny Depp calls for death...,"[""liberals""]","[""miscontrue things"", ""take things in a weird ...",1.0,0
3,\nDavid Knight‏ \n\nNotice how quickly things ...,[],[],0.0,1
4,\nFinland fireball: Time-lapse video shows nig...,[],[],NaN,1
...,...,...,...,...,...
35498,👉 Illegally in the country after 5 deportation...,[],[],NaN,1
35500,💥Breaking💥\nJulian Assange is the gate keeper ...,[],[],0.0,1
35501,📖 2Kings 22:19 because your heart was peniten...,[],[],NaN,1
35502,🚨#FAKENEWSAWARDS🚨\n\n🚨 who is #1 fake news ?🚨\...,[],[],0.0,1


In [695]:
training_csv

,post,group,implications,whoTarget,offensiveYN
0,"\n\nBill Kristol and Ben Shaprio, two turds in...",[],[],0.0,1
1,\n\nRose\n🌹Taylor‏ @RealRoseTaylor 6h6 hours a...,[],[],NaN,1
2,\nCharlie Kirk‏\n\nJohnny Depp calls for death...,"[""liberals""]","[""miscontrue things"", ""take things in a weird ...",1.0,0
3,\nDavid Knight‏ \n\nNotice how quickly things ...,[],[],0.0,1
4,\nFinland fireball: Time-lapse video shows nig...,[],[],NaN,1
...,...,...,...,...,...
35498,👉 Illegally in the country after 5 deportation...,[],[],NaN,1
35500,💥Breaking💥\nJulian Assange is the gate keeper ...,[],[],0.0,1
35501,📖 2Kings 22:19 because your heart was peniten...,[],[],NaN,1
35502,🚨#FAKENEWSAWARDS🚨\n\n🚨 who is #1 fake news ?🚨\...,[],[],0.0,1


In [696]:
#training_csv["group"] = training_csv["group"].apply(lambda label: "NaN" if label=='[]' else label)
training_csv["offensiveYN"] = training_csv["offensiveYN"].apply(lambda label: "[OffN]" if label==1 else "[OffY]")

In [697]:
print(training_csv["offensiveYN"])

0        [OffN]
1        [OffN]
2        [OffY]
3        [OffN]
4        [OffN]
          ...  
35498    [OffN]
35500    [OffN]
35501    [OffN]
35502    [OffN]
35503    [OffY]
Name: offensiveYN, Length: 35424, dtype: object


In [698]:
training_csv["post"] = training_csv["post"].apply(lambda label: "[BOS]" + label + "[CLS]")

In [699]:
training_csv.dtypes

post             object
group            object
implications     object
whoTarget       float64
offensiveYN      object
dtype: object

In [700]:
def createNewLine(i, group):
    copy_df = training_csv.copy(deep=True)
    copy_df.append(copy_df.loc[[i]])
    copy_df.loc[[len(training_csv)-1]]["implications"] = group[len(group)-1]
    del group[-1]
    copy_df.loc[i,"implications"] = str(group)
    return copy_df

In [701]:
import ast
#one implication per line
# if > 1 implication, copy entire line, put on end, change group
list_implications = []
list_group = []
for i in training_csv.index:
    list_val = ast.literal_eval(training_csv.loc[[i]]["implications"].values[0])
    if (len(list_val) > 1):
        while (len(list_val) > 1):
            training_csv = createNewLine(i, list_val)
            list_val = ast.literal_eval(training_csv.loc[[i]]["implications"].values[0])
    list_group.append(",".join(map(str, ast.literal_eval(training_csv.loc[[i]]["group"].values[0]))))

KeyboardInterrupt: 

In [ ]:
#training_csv["implications"] = list_implications
training_csv["group"] = list_group

In [84]:
training_csv
#do final pass over the implications and get rid of list format
for i in training_csv.index:
    list_implications.append(",".join(map(str, ast.literal_eval(training_csv.loc[[i]]["implications"].values[0]))))

In [85]:
training_csv["implications"] = list_implications
training_csv

,post,group,implications,whoTarget,offensiveYN
0,"\n\nBill Kristol and Ben Shaprio, two turds in...",,,0.0,1
1,\n\nRose\n🌹Taylor‏ @RealRoseTaylor 6h6 hours a...,,,NaN,1
2,\nCharlie Kirk‏\n\nJohnny Depp calls for death...,liberals,miscontrue things,1.0,0
3,\nDavid Knight‏ \n\nNotice how quickly things ...,,,0.0,1
4,\nFinland fireball: Time-lapse video shows nig...,,,NaN,1
...,...,...,...,...,...
35498,👉 Illegally in the country after 5 deportation...,,,NaN,1
35500,💥Breaking💥\nJulian Assange is the gate keeper ...,,,0.0,1
35501,📖 2Kings 22:19 because your heart was peniten...,,,NaN,1
35502,🚨#FAKENEWSAWARDS🚨\n\n🚨 who is #1 fake news ?🚨\...,,,0.0,1


In [248]:
len(training_csv["implications"][0])

0

In [249]:
final_list = []
for i in training_csv.index:
    if (training_csv.loc[[i]]["whoTarget"].values[0] == 0.0 and training_csv.loc[[i]]["offensiveYN"].values[0] == "[OffY]"):
        final_list.append("[IND] [END]")
    elif (training_csv.loc[[i]]["offensiveYN"].values[0] == "[OffN]"):
        final_list.append(" [END]")
    else:
        final_list.append("[GRP] " + str(training_csv.loc[[i]]["group"].values[0]))# + " [STE] " + str(training_csv.loc[[i]]["implications"].values[0]) + " [END]")

In [250]:
training_csv["implications"] = training_csv["implications"].apply(lambda label: "[STE] " + label + " [END]" if len(label)>0 else label)

In [251]:
training_csv["group"] = final_list
training_csv = training_csv.drop(["whoTarget"], axis=1)

[BOS] I hate all gay cause they are bad [CLS] [OffY] [GRP] gay men [STE] implies that gays are bad [END] \
[BOS] You're a moron [CLS] [OffY] [IND] [END] \
[BOS] I cannot stand women sport announcers [CLS] [OffY] [GRP] Women [STE] Women aren't knowledgeable about sports [END]

post [CLS] offensiveYN [GRP] group [STE] implications [END]

# scramble method

In [252]:
final_scramble_output = []
final_scramble_input = []

In [253]:
training_csv.loc[[2]].values.tolist()

[['[BOS]""" I have no problem being called Benny! Just don\'t call my Ben 10. Or else You\'ll meet with a terrible fate. You cocky bitch.. "" #Bot"[CLS]',
  ' [END]',
  '',
  '[OffN]']]

### When scrambling, have the same posts multiple times with different inputs outputs scrambled

In [254]:
len(training_csv)

4666

In [255]:
import random 
for i in range(0, len(training_csv.index)):
    # create 4 versions of each post in scrambled order (more data)
    for i in range(0,4):
        # create a list of all things to append to either input or output in the dataframe, append whatevers left to output
        toAppend = list(filter(lambda item: item.strip(), training_csv.loc[[i]].values.tolist()[0]))
        # randomly decide how much is appended to the output and how much for the input (at least one thing has to be in the input/output
        appendNum = random.randint(1, len(toAppend)-1)
        # loop by appendnum
        # new list er input and output, make into a sentence later
        listApp = []
        for i in range(0, appendNum) :
            # randomly decide what to append to the output and what to append to the input
            i = random.randint(0, len(toAppend)-1)
            value = toAppend.pop(i)
            listApp.append(value)
        final_scramble_input.append(listApp)
        # whatever is left goes to output
        final_scramble_output.append(toAppend)

In [256]:
len(final_scramble_input) == 4 * len(training_csv)

True

In [257]:
final_scramble_input[0]
final_scramble_output[0]

['[BOS] \nRed Nation Rising\n\u200f\n\n\xa0\n@RedNationRising\n \nMYSTERY: Awan Brothers Court Date Disappears From Calendar After Being Postponed Twice[CLS]',
 '[OffN]']

In [258]:
# list comprehension - make each entry into one string
string_scramble_input = ["".join(f).strip() for f in final_scramble_input]
string_scramble_output = ["".join(f).strip() for f in final_scramble_output]

In [259]:
string_scramble_input[0]

'[END]'

In [260]:
final_df = pd.DataFrame()

In [261]:
final_df["input"] = string_scramble_input
final_df["output"] = string_scramble_output

In [360]:
string_scramble_input[0:5]

['[END]',
 '[END]',
 '[BOS]""" I have no problem being called Benny! Just don\'t call my Ben 10. Or else You\'ll meet with a terrible fate. You cocky bitch.. "" #Bot"[CLS] [END]',
 '[BOS]""" animal friend in My ass"" You need some Help @realDonaldTrump &#128548;&#128548;&#128548;&#128548; #SaveAlaskanBear FROM FUCKING TRUMP. https://t.co/7ADxKIILUs"[CLS]',
 '[OffN] [END]']

In [361]:
string_scramble_output[0:5]

['[BOS] \nRed Nation Rising\n\u200f\n\n\xa0\n@RedNationRising\n \nMYSTERY: Awan Brothers Court Date Disappears From Calendar After Being Postponed Twice[CLS][OffN]',
 '[BOS] they found marijuana, butane hash oil (BHO) also known as “marijuana butter”, Xanax pills, digital scales with meth residue, plastic baggies, over 10 grams of methamphetamine and over $600 in cash[CLS][OffN]',
 '[OffN]',
 '[END][OffN]',
 '[BOS] \nRed Nation Rising\n\u200f\n\n\xa0\n@RedNationRising\n \nMYSTERY: Awan Brothers Court Date Disappears From Calendar After Being Postponed Twice[CLS]']

In [263]:
final_df.to_csv("SBIC.dev.scrambled.csv", index=False)

In [27]:
pd.read_csv("")

,input,output
0,[BOS] \n if you make a fact filled book or boo...,[OffN] [END]
1,"[BOS] \nAlex Jones & Mike Cernovich: ""It's cra...",[OffN] [END]
2,[BOS] \nJason Donner‏:\n\nFBI Deputy Director...,[OffN] [END]
3,[BOS] happy birthday #LittleRocketMan\nyou #D...,[OffN] [END]
4,"[BOS] "" is your name kim jong - un , m'boy ? ""...",[OffN] [END]
...,...,...
995,[BOS] Gshit though I can't stand a nigga who c...,[OffY] [GRP] black folks [STE] black men are b...
996,[BOS] Gully Foyle is my name \nAnd Terra is my...,[OffN] [END]
997,"[BOS] Guys ( and gals ) , just wanted to say :...",[OffY] [GRP] black folks [STE] black folks are...
998,[BOS] Guys I have a question: when a gay coupl...,"[OffY] [GRP] gay men,gays [STE] gay people are..."


In [4]:
# from https://homes.cs.washington.edu/~msap/pdfs/sap2020socialbiasframes.pdf and 
# https://arxiv.org/pdf/2109.05322v1.pdf
sample_input = []
samples = ["Kendrick just dropped the coldest video of all fucking time", "What a fucking stupid bitch shut the fuck up already",
           "need to fuck sum bad , I got some pressure built up :P", "I take a bitch home give her long dick", 
           "You annoyed me every day for months you’re a fucking moron", 
           "My problem with Korean artists: I dont know how to pronounce your name I can’t hanggul to save my life",
           "I’ll continue this thread till non-lifting fags stop lifting like idiots at the gym... Which is never.",
           "I thought drugs were the only things black people could shoot up Boy was I wrong",
           "white revolution is the only solution",
           "yes you are fine in a white majority country. how is mexico doing these days? people come here because you couldn’t build it.",
           "black people only seem to thrive in white countries. how strange", 
           "soon white people will not be a majority anywhere. you believe",
           "muslims are always wanting to kill someone! its their duty to kill the infidel.",
           "oh how i wish a man with a gun was there! wake up white people! they hate us"
        ]
for sample in samples:
    sample_input.append("[BOS] " + sample + " [CLS] ")

In [5]:
sample_input

['[BOS] Kendrick just dropped the coldest video of all fucking time [CLS] ',
 '[BOS] What a fucking stupid bitch shut the fuck up already [CLS] ',
 '[BOS] need to fuck sum bad , I got some pressure built up :P [CLS] ',
 '[BOS] I take a bitch home give her long dick [CLS] ',
 '[BOS] You annoyed me every day for months you’re a fucking moron [CLS] ',
 '[BOS] My problem with Korean artists: I dont know how to pronounce your name I can’t hanggul to save my life [CLS] ',
 '[BOS] I’ll continue this thread till non-lifting fags stop lifting like idiots at the gym... Which is never. [CLS] ',
 '[BOS] I thought drugs were the only things black people could shoot up Boy was I wrong [CLS] ',
 '[BOS] white revolution is the only solution [CLS] ',
 '[BOS] yes you are fine in a white majority country. how is mexico doing these days? people come here because you couldn’t build it. [CLS] ',
 '[BOS] black people only seem to thrive in white countries. how strange [CLS] ',
 '[BOS] soon white people will 

In [6]:
sample_df = pd.DataFrame()
sample_df["input"] = sample_input


In [7]:
sample_df.to_csv("samples.csv", index=False)

,input
0,[BOS] Kendrick just dropped the coldest video ...
1,[BOS] What a fucking stupid bitch shut the fuc...
2,"[BOS] need to fuck sum bad , I got some pressu..."
3,[BOS] I take a bitch home give her long dick [...
4,[BOS] You annoyed me every day for months you’...
5,[BOS] My problem with Korean artists: I dont k...
6,[BOS] I’ll continue this thread till non-lifti...
7,[BOS] I thought drugs were the only things bla...
8,[BOS] white revolution is the only solution [C...
9,[BOS] yes you are fine in a white majority cou...


# Accuracy scores

- draw conclusions about what needs to happen, what are pitfalls/missing, more data needed? Which of the categories from latent hate they are (best guess, pick samples to show them)
- look at stereotypes, and guess what the model may need
- create demo to help with playing with it
- grab data that has already been released
- dynahate (subtle hatred)
- retrain group and append group to input and only guess stereotype
- input and output is whole string, take fraction as input and remain as output, train model on all forms of input (when using at inference time, it generates what you want)
- scramble method (try both models, see what happens)
- take examples from social boas, implicit hate, dynsahate, see what the model generates/compare to actual data, derive insights from this
- paper that maarten shared (they - how would model pick up on it)
- read papers in chat

In [355]:
from sklearn.metrics import f1_score
import re

In [41]:
y_test = pd.read_csv("SBIC.dev.1.csv")["output"]
#y_test = y_test.drop(["dataSource", "offensiveYN", "sexYN", "intentYN", "targetCategory"], axis=1, index=None)
#y_test.columns = ["post", "group", "implications", "whoTarget", "offensiveYN"]
#y_pred = pd.read_csv("predictions_scrambled.csv", index_col=0)["Generated Text"].values #TODO: fix path if necessary

In [43]:
y_test

0                                      [boo] [OffN] [eoo]
1                                      [boo] [OffN] [eoo]
2                                      [boo] [OffN] [eoo]
3                                      [boo] [OffN] [eoo]
4                                      [boo] [OffN] [eoo]
                              ...                        
4661                                   [boo] [OffN] [eoo]
4662    [boo] [OffY] [grp] black folks, asian folks, l...
4663                                   [boo] [OffN] [eoo]
4664    [boo] [OffY] [grp] jewish folks [ste] all jews...
4665                                   [boo] [OffN] [eoo]
Name: output, Length: 4666, dtype: object

In [44]:
# use reg expressions to split into offy, group, etc. - calculate accuracy on each
# tokens = ['[BOS]', '[CLS]', '[OFFY]', '[OFFN]', '[IND]', '[STE]', '[END]', '[GRP]']
res = []
# split on parenthesis
for i in range(0, len(y_test)):
    res.append(list(filter(lambda x: x != "", [sentence.strip() for sentence in
                                                    y_test[i].replace("<pad>", '').replace("[", '').replace("boo",'')
                                                    .replace("grp", '').replace("ste", '').replace("eoo", '')
                                                    .split(']')])))
   # res.append(list(filter(lambda x: x != "", [sentence.strip() for sentence in y_pred[i].replace("<pad>", '').replace("[", '').replace("END", '').replace("STE", '').replace("GRP", '').split(']')])))

In [353]:
    #TODO: add all characters to re split re.split("[]+")
# binary acc for offn/y, ignore group/implication if OffN
res = pd.DataFrame(res)
res[0] = res[0].apply(lambda label: 1 if label=="OffN" else 0)

In [ ]:
res

In [275]:
sum_int = sum(res[0].values == y_test["offensiveYN"])

# Accuracy of OFFy vs OFFn

In [276]:
sum_int/len(res[0].values == y_test["offensiveYN"])

0.845049292756108

# F1 Score of OFFy vs OFFn

In [277]:
f1_score(y_test["offensiveYN"].values, res[0].values)

0.8708697981782461

In [292]:
res[1]

0                                          None
1                                          None
2                                          None
3                                          None
4                                          None
                         ...                   
4661                                       None
4662    black folks, asian folks, latino/latina
4663                                       None
4664                               jewish folks
4665                                       None
Name: 1, Length: 4666, dtype: object

# Qualitative Analysis - how does the models groupings relate to the original (maybe use contains?)

In [290]:
# does it match any of the groups from the original 
# ngram - BLEU score nltk package to use 
# exact match and bleu score
match = []
for i in range(0,len(res)):
    if (list(y_test["offensiveYN"].values)[i] == 0):
        res_val = str(list(res[1].values)[i])
        y_test_val = list(y_test["group"].values)
        match.append(any(res_val in s for s in y_test_val))

{False, True}

In [293]:
sum(match) / len(match)

0.7109634551495017

# New Scrambler Method as Per last meeting
- need to overrepresent offensive samples and have more structure with the tokens so the model learns
- create each kind of scramble for each input
#### Most important:
1) generate stereotype given just post
2) generate stereotype given post, group, and offensiveness
3) generate post given group, sterotype, and offensiveness

Making three kinds of scrambles:
1) regular, with new tokens
2) full scramble, with new tokens
3) important scramble, with new tokens

In [105]:
training_csv = pd.read_csv("SBIC.v2.agg.dev.csv", index_col=0)

In [106]:
training_csv

,post,targetMinority,targetCategory,targetStereotype,whoTarget,intentYN,sexYN,offensiveYN,dataSource,hasBiasedImplication
0,\nRed Nation Rising\n‏\n\n \n@RedNationRising...,[],[],[],NaN,0.110000,0.000000,0.000000,Gab,1
1,"they found marijuana, butane hash oil (BHO) a...",[],[],[],0.0,0.110000,0.000000,0.166667,Gab,1
2,""""""" I have no problem being called Benny! Just...",[],[],[],0.0,0.553333,0.000000,0.500000,t/founta,1
3,""""""" animal friend in My ass"""" You need some He...",[],[],[],0.0,0.886667,0.166667,1.000000,t/founta,1
4,"""""""A mans belly shall be satisfied with the fr...",[],[],[],0.0,0.000000,0.000000,0.000000,t/founta,1
...,...,...,...,...,...,...,...,...,...,...
4668,“One special advantage of the skeptical attitu...,[],[],[],0.0,0.000000,0.000000,0.000000,t/founta,1
4669,“The only thing worse than living in a country...,"[""black folks, asian folks, latino/latina folk...","[""race""]","[""non-white people ruin the life of white peop...",1.0,1.000000,0.000000,1.000000,Gab,0
4670,“Two things are infinite: the universe and hum...,[],[],[],0.0,0.220000,0.000000,0.166667,Gab,1
4671,♪ &#9835; ♬ believe it or not i 'm gassing the...,"[""jewish folks""]","[""culture""]","[""all jews have big noses"", ""died in the holoc...",1.0,1.000000,0.000000,1.000000,r/meanjokes,0


In [107]:
training_csv = training_csv.drop(["dataSource", "offensiveYN", "sexYN", "intentYN", "targetCategory"], axis=1, index=None)
training_csv.columns = ["post", "group", "implications", "whoTarget", "offensiveYN"]
training_csv

,post,group,implications,whoTarget,offensiveYN
0,\nRed Nation Rising\n‏\n\n \n@RedNationRising...,[],[],NaN,1
1,"they found marijuana, butane hash oil (BHO) a...",[],[],0.0,1
2,""""""" I have no problem being called Benny! Just...",[],[],0.0,1
3,""""""" animal friend in My ass"""" You need some He...",[],[],0.0,1
4,"""""""A mans belly shall be satisfied with the fr...",[],[],0.0,1
...,...,...,...,...,...
4668,“One special advantage of the skeptical attitu...,[],[],0.0,1
4669,“The only thing worse than living in a country...,"[""black folks, asian folks, latino/latina folk...","[""non-white people ruin the life of white peop...",1.0,0
4670,“Two things are infinite: the universe and hum...,[],[],0.0,1
4671,♪ &#9835; ♬ believe it or not i 'm gassing the...,"[""jewish folks""]","[""all jews have big noses"", ""died in the holoc...",1.0,0


In [108]:
#training_csv["group"] = training_csv["group"].apply(lambda label: "NaN" if label=='[]' else label)
training_csv["offensiveYN"] = training_csv["offensiveYN"].apply(lambda label: "[OffN]" if label==1 else "[OffY]")

In [109]:
def createNewLine(i, group):
    copy_df = training_csv.copy(deep=True)
    copy_df.append(copy_df.loc[[i]])
    copy_df.loc[[len(training_csv)-1]]["implications"] = group[len(group)-1]
    del group[-1]
    copy_df.loc[i,"implications"] = str(group)
    return copy_df

In [110]:
import ast
#one implication per line
# if > 1 implication, copy entire line, put on end, change group
list_implications = []
list_group = []
for i in training_csv.index:
    list_val = ast.literal_eval(training_csv.loc[[i]]["implications"].values[0])
    if (len(list_val) > 1):
        while (len(list_val) > 1):
            training_csv = createNewLine(i, list_val)
            list_val = ast.literal_eval(training_csv.loc[[i]]["implications"].values[0])
    list_group.append(",".join(map(str, ast.literal_eval(training_csv.loc[[i]]["group"].values[0]))))

In [111]:
#training_csv["implications"] = list_implications
training_csv["group"] = list_group

In [112]:
training_csv
#do final pass over the implications and get rid of list format
for i in training_csv.index:
    list_implications.append(",".join(map(str, ast.literal_eval(training_csv.loc[[i]]["implications"].values[0]))))

In [113]:
training_csv["implications"] = list_implications

## 1) Regular

# 50/50 dataframe

In [92]:
training_csv

,post,group,implications,whoTarget,offensiveYN
0,"\n\nBill Kristol and Ben Shaprio, two turds in...",,,0.0,[OffN]
1,\n\nRose\n🌹Taylor‏ @RealRoseTaylor 6h6 hours a...,,,NaN,[OffN]
2,\nCharlie Kirk‏\n\nJohnny Depp calls for death...,liberals,miscontrue things,1.0,[OffY]
3,\nDavid Knight‏ \n\nNotice how quickly things ...,,,0.0,[OffN]
4,\nFinland fireball: Time-lapse video shows nig...,,,NaN,[OffN]
...,...,...,...,...,...
35498,👉 Illegally in the country after 5 deportation...,,,NaN,[OffN]
35500,💥Breaking💥\nJulian Assange is the gate keeper ...,,,0.0,[OffN]
35501,📖 2Kings 22:19 because your heart was peniten...,,,NaN,[OffN]
35502,🚨#FAKENEWSAWARDS🚨\n\n🚨 who is #1 fake news ?🚨\...,,,0.0,[OffN]


In [93]:
offY_df = training_csv.where(training_csv["offensiveYN"] == "[OffY]").dropna(subset=['offensiveYN'])
num_offY = len(offY_df.index)
offN_df = training_csv.where(training_csv["offensiveYN"] == "[OffN]").dropna(subset=['offensiveYN']).sample(n=num_offY).reset_index()
fifty_df = pd.concat([offY_df, offN_df])
fifty_df = fifty_df.sample(frac=1).reset_index(drop=True)

In [101]:
fifty_df

,post,group,implications,whoTarget,offensiveYN,index
0,What do work boots and slaves have in common? ...,black folks,are second class citizens,1.000000,[OffY],NaN
1,&#128557;&#128557; RT @KingHorseDick: Welp RT ...,women,are bitches,1.000000,[OffY],NaN
2,Mommy why is my backpack so heavy? ALLAHU AKBA...,muslim folks,muslims are terrorists,0.666667,[OffY],NaN
3,There should have been a Hulk episode where Da...,,,0.000000,[OffN],22957.0
4,Out of Pravana blue. Guess I'm stuck with mixi...,,,NaN,[OffN],18732.0
...,...,...,...,...,...,...
24247,R.I.P. Stephen Hawkings We lost another wheel ...,,,0.000000,[OffN],19258.0
24248,Whore making nu-male cuck friend write an essa...,women,women are only sex objects,1.000000,[OffY],NaN
24249,"REMINDER: Even in a woman's deepest, darkest d...",women,don't really have depression,1.000000,[OffY],NaN
24250,A rock and an emo kid falls off a tree who hit...,,,0.000000,[OffN],3578.0


In [96]:
training_csv = fifty_df

In [97]:
final_input = []
final_output = []
for i in training_csv.index:
    final_input.append("[boi] " + str(training_csv.loc[[i]]["post"].values[0]) + " [eoi]")
    if (training_csv.loc[[i]]["whoTarget"].values[0] == 0.0 and training_csv.loc[[i]]["offensiveYN"].values[0] == "[OffY]"):
        print("here")
        final_output.append("[boo] " + str(training_csv.loc[[i]]["offensiveYN"].values[0]) + "[ind] [eoo]")
    elif (training_csv.loc[[i]]["offensiveYN"].values[0] == "[OffN]"):
        final_output.append("[boo] " + str(training_csv.loc[[i]]["offensiveYN"].values[0]) + " [eoo]")
    else:
        final_output.append("[boo] " + str(training_csv.loc[[i]]["offensiveYN"].values[0]) + " [grp] " + 
                          str(training_csv.loc[[i]]["group"].values[0]) + " [ste] " + 
                          str(training_csv.loc[[i]]["implications"].values[0]) + " [eoo]")

here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here


In [98]:
regular_df = pd.DataFrame()

In [99]:
regular_df["input"] = final_input
regular_df["output"] = final_output

In [100]:
regular_df

,input,output
0,[boi] What do work boots and slaves have in co...,[boo] [OffY] [grp] black folks [ste] are secon...
1,[boi] &#128557;&#128557; RT @KingHorseDick: We...,[boo] [OffY] [grp] women [ste] are bitches [eoo]
2,[boi] Mommy why is my backpack so heavy? ALLAH...,[boo] [OffY] [grp] muslim folks [ste] muslims ...
3,[boi] There should have been a Hulk episode wh...,[boo] [OffN] [eoo]
4,[boi] Out of Pravana blue. Guess I'm stuck wit...,[boo] [OffN] [eoo]
...,...,...
24247,[boi] R.I.P. Stephen Hawkings We lost another ...,[boo] [OffN] [eoo]
24248,[boi] Whore making nu-male cuck friend write a...,[boo] [OffY] [grp] women [ste] women are only ...
24249,"[boi] REMINDER: Even in a woman's deepest, dar...",[boo] [OffY] [grp] women [ste] don't really ha...
24250,[boi] A rock and an emo kid falls off a tree w...,[boo] [OffN] [eoo]


In [102]:
regular_df.to_csv("SBIC.trn.2.5050.csv", index=False)

In [122]:
predictions_df = pd.read_csv("predictions_regular.csv", index_col=0)

In [123]:
predictions_df

,Source Text,Generated Text
0,[boi] \nRed Nation Rising\n‏\n\n \n@RedNation...,</s>
1,"[boi] they found marijuana, butane hash oil (...",</s>
2,"[boi] """""" I have no problem being called Benny...",</s>
3,"[boi] """""" animal friend in My ass"""" You need s...",</s>
4,"[boi] """"""A mans belly shall be satisfied with ...",</s>
...,...,...
4661,[boi] “One special advantage of the skeptical ...,</s>
4662,[boi] “The only thing worse than living in a c...,</s>
4663,[boi] “Two things are infinite: the universe a...,</s>
4664,[boi] ♪ &#9835; ♬ believe it or not i 'm gassi...,</s>


## 2) Full Scramble

In [97]:
final_input = []
final_output = []
for i in training_csv.index:
    # If it isn't offensive, do the usual (from "regular" scramble)
    if (training_csv.loc[[i]]["offensiveYN"].values[0] == "[OffN]"):
        final_input.append("[boi] " + "[bos] " + str(training_csv.loc[[i]]["post"].values[0]) + " [cls]" + " [eoi]")
        final_output.append("[boo] " + str(training_csv.loc[[i]]["offensiveYN"].values[0]) + " [eoo]")
    # Otherwise, scramble up the input and output according to what is above
    elif (training_csv.loc[[i]]["whoTarget"].values[0] == 0.0 and training_csv.loc[[i]]["offensiveYN"].values[0] == "[OffY]"):
        # Case 1: given offensiveness, group, and stereotype, get post
        final_input.append("[boi] " + str(training_csv.loc[[i]]["offensiveYN"].values[0]) + "[ind] [eoi]") 
        final_output.append("[boo] " + "[bos] " + str(training_csv.loc[[i]]["post"].values[0]) + " [cls]" + " [eoo]")
        # Case 3: given offensiveness, generate post, stereotype, and group
        final_input.append("[boi] " + str(training_csv.loc[[i]]["offensiveYN"].values[0]) + " [eoi]") 
        final_output.append("[boo] " + "[bos] " + str(training_csv.loc[[i]]["post"].values[0]) + " [cls] " + "[ind] [eoo]")
        # Case 4: regular 
        final_input.append("[boi] " + "[bos] " + str(training_csv.loc[[i]]["post"].values[0]) + " [cls]" + " [eoi]")
        final_output.append("[boo] " + str(training_csv.loc[[i]]["offensiveYN"].values[0]) + "[ind] [eoo]")
    else:
        # Case 1: given offensiveness, group, and stereotype, get post
        final_input.append("[boi] " + str(training_csv.loc[[i]]["offensiveYN"].values[0]) + " [grp] " + 
                          str(training_csv.loc[[i]]["group"].values[0]) + " [ste] " + 
                          str(training_csv.loc[[i]]["implications"].values[0]) + " [eoi]")
        final_output.append("[boo] " + "[bos] " + str(training_csv.loc[[i]]["post"].values[0]) + " [cls]" + " [eoo]")
        # Case 2: given offensiveness and group, generate stereotype and post
        final_input.append("[boi] " + str(training_csv.loc[[i]]["offensiveYN"].values[0]) + " [grp] " + 
                          str(training_csv.loc[[i]]["group"].values[0]) + " [eoi]")
        final_output.append("[boo] " + "[bos] " + str(training_csv.loc[[i]]["post"].values[0]) + " [cls]" + " [ste] " + 
                          str(training_csv.loc[[i]]["implications"].values[0]) + " [eoo]") 
        # Case 3: given offensiveness, generate post, stereotype, and group
        final_input.append("[boi] " + str(training_csv.loc[[i]]["offensiveYN"].values[0]) + " [eoi]")
        final_output.append("[boo] [bos]" + str(training_csv.loc[[i]]["post"].values[0]) + " [cls] [grp] " + 
                          str(training_csv.loc[[i]]["group"].values[0]) + " [ste] " + 
                          str(training_csv.loc[[i]]["implications"].values[0]) + " [eoo]")         
        # Case 4: regular
        final_input.append("[boi] " + "[bos] " + str(training_csv.loc[[i]]["post"].values[0]) + " [cls]" + " [eoi]")
        final_output.append("[boo] " + str(training_csv.loc[[i]]["offensiveYN"].values[0]) + " [grp] " + 
                          str(training_csv.loc[[i]]["group"].values[0]) + " [ste] " + 
                          str(training_csv.loc[[i]]["implications"].values[0]) + " [eoo]")

In [ ]:
final_output[0:150]

In [98]:
full_scramble_df = pd.DataFrame()

In [99]:
full_scramble_df["input"] = final_input
full_scramble_df["output"] = final_output

In [100]:
full_scramble_df

,input,output
0,[boi] [bos] \nRed Nation Rising\n‏\n\n \n@Red...,[boo] [OffN] [eoo]
1,"[boi] [bos] they found marijuana, butane hash...",[boo] [OffN] [eoo]
2,"[boi] [bos] """""" I have no problem being called...",[boo] [OffN] [eoo]
3,"[boi] [bos] """""" animal friend in My ass"""" You ...",[boo] [OffN] [eoo]
4,"[boi] [bos] """"""A mans belly shall be satisfied...",[boo] [OffN] [eoo]
...,...,...
10077,[boi] [OffY] [grp] jewish folks [ste] all jews...,[boo] [bos] ♪ &#9835; ♬ believe it or not i 'm...
10078,[boi] [OffY] [grp] jewish folks [eoi],[boo] [bos] ♪ &#9835; ♬ believe it or not i 'm...
10079,[boi] [OffY] [eoi],[boo] [bos]♪ &#9835; ♬ believe it or not i 'm ...
10080,[boi] [bos] ♪ &#9835; ♬ believe it or not i 'm...,[boo] [OffY] [grp] jewish folks [ste] all jews...


In [101]:
full_scramble_df.to_csv("full.scramble.dev.csv", index=False)

In [27]:
pd.read_csv("")

,input,output
0,[BOS] \n if you make a fact filled book or boo...,[OffN] [END]
1,"[BOS] \nAlex Jones & Mike Cernovich: ""It's cra...",[OffN] [END]
2,[BOS] \nJason Donner‏:\n\nFBI Deputy Director...,[OffN] [END]
3,[BOS] happy birthday #LittleRocketMan\nyou #D...,[OffN] [END]
4,"[BOS] "" is your name kim jong - un , m'boy ? ""...",[OffN] [END]
...,...,...
995,[BOS] Gshit though I can't stand a nigga who c...,[OffY] [GRP] black folks [STE] black men are b...
996,[BOS] Gully Foyle is my name \nAnd Terra is my...,[OffN] [END]
997,"[BOS] Guys ( and gals ) , just wanted to say :...",[OffY] [GRP] black folks [STE] black folks are...
998,[BOS] Guys I have a question: when a gay coupl...,"[OffY] [GRP] gay men,gays [STE] gay people are..."


# MORE DATA

'acl.id' is the unique ID of the entry.

'Text' is the content which has been entered. All content is synthetic.

'Label' is a binary variable, indicating whether or not the content has been identified as hateful. It takes two values: hate, nothate.

'Type' is a categorical variable, providing a secondary label for hateful content. For hate it can take five values: Animosity, Derogation, Dehumanization, Threatening and Support for Hateful Entities. Please see the paper for more detail. For nothate the 'type' is 'none'. In round 1 the 'type' was not given and is marked as 'notgiven'.

'Target' is a categorical variable, providing the group that is attacked by the hate. It can include intersectional characteristics and multiple groups can be identified. For nothate the type is 'none'. Note that in round 1 the 'target' was not given and is marked as 'notgiven'.

'Level' reports whether the entry is original content or a perturbation.

'Round' is a categorical variable. It gives the round of data entry (1, 2, 3 or 4) with a letter for whether the entry is original content ('a') or a perturbation ('b'). Perturbations were not made for round 1.

'Round.base' is a categorical variable. It gives the round of data entry, indicated with just a number (1, 2, 3 or 4).

'Split' is a categorical variable. it gives the data split that the entry has been assigned to. This can take the values 'train', 'dev' and 'test'. The choice of splits is explained in the paper.

'Annotator' is a categorical variable. It gives the annotator who entered the content. Annotator IDs are random alphanumeric strings. There are 20 annotators in the dataset.

'acl.id.matched' is the ID of the matched entry, connecting the original (given in 'acl.id') and the perturbed version.

For identities (recorded under 'Target') we use shorthand labels to constructed the dataset, which can be converted (and grouped) as follows:

In [774]:
import pandas as pd
c=pd.read_csv("https://raw.githubusercontent.com/bvidgen/Dynamically-Generated-Hate-Speech-Dataset/main/Dynamically%20Generated%20Hate%20Dataset%20v0.2.2.csv", index_col=0)
c.drop(["acl.id", "X1", "acl.id.matched"], axis=1)
# get dynahate model into formal to test our model on (predict), calculate acc
# columns for training_csv: post, implications, offy/n, whoTarget, group
# columns to use for DynaHate : text (post), label (offy/n), target (group), (no stereotypes? See how model functions)

,text,label,type,target,level,split,round.base,annotator,round
1,dalits are lowlives,hate,notgiven,notgiven,original,train,1,GNZuCtwed3,1
2,gay people are a burden to our society,hate,notgiven,notgiven,original,train,1,GNZuCtwed3,1
3,Arabs are not welcome,hate,notgiven,notgiven,original,train,1,vDe7GN0NrL,1
4,I'm not saying we should actually eliminate he...,hate,notgiven,notgiven,original,train,1,oemYWm1Tjg,1
5,bananas are for black people,hate,notgiven,notgiven,original,test,1,QiOKkCi7F8,1
...,...,...,...,...,...,...,...,...,...
41251,I hate drags lol,nothate,none,none,original,test,1,GNZuCtwed3,1
41252,My washing machine is bitch,nothate,none,none,original,test,1,vDe7GN0NrL,1
41253,Fulham is just a bunch of losers,nothate,none,none,original,test,1,TrRF46JWfP,1
41254,K I L L the T R A N S,hate,notgiven,notgiven,original,test,1,oemYWm1Tjg,1


In [775]:
# none but if its offensive
# round 2 and above only 
c = c[(c["round"] != "1")]
c

,acl.id,X1,text,label,type,target,level,split,round.base,annotator,round,acl.id.matched
1290,acl11158,1290,There are too many costas in this country,nothate,none,none,perturbation,train,2,GNZuCtwed3,2b,acl20309
1291,acl11159,1291,There is way too many m0ng0l0id kids at my sch...,hate,derogation,dis,original,train,2,TrRF46JWfP,2a,acl19858
1293,acl11160,1293,I called my brother a twat and my mom got so m...,nothate,none,none,perturbation,train,2,TrRF46JWfP,2b,acl19997
1294,acl11161,1294,the worst of them are the taaaaaaaaaards becau...,hate,derogation,dis,original,train,2,TrRF46JWfP,2a,acl11162
1295,acl11162,1295,the worst of them are the viruses because they...,nothate,none,none,perturbation,train,2,LqLKTtrOmx,2b,acl11161
...,...,...,...,...,...,...,...,...,...,...,...,...
34727,acl41251,34727,I have that opinion. absolutely everyone I act...,nothate,none,none,perturbation,dev,4,dqrONtdjbt,4b,acl34594
34728,acl41252,34728,"This dark female, I am not going to use the hu...",hate,dehumanization,bla.wom,original,dev,4,CAgNlUizNm,4a,acl35570
34729,acl41253,34729,I guess you are assuming that the french laws ...,hate,derogation,immig,perturbation,dev,4,TbUBpfn6iP,4b,acl35966
34730,acl41254,34730,"I mean, theres no one better right now to give...",nothate,none,none,original,dev,4,GNZuCtwed3,4a,acl36963


In [776]:
c["label"] = c["label"].apply(lambda label: 1 if label=="nothate" else 0)

In [777]:
c = c[c['target'].map(c['target'].value_counts()) >= 200].reset_index()

In [778]:
#clean up groupings to be what model is used to from SBIC
c["target"] = c["target"].map({"notgiven" : "notgiven", "wom":"women", "bla":"black", "jew":"jewish", "mus":"muslim", "immig":"immigrants", "asi.south":"south asians", "none" : "none", "transgender" : "transgender people", "ref":"refugee", "arab":"arabic people", "indig" : "indigenous", "mixed.race" : "mixed people", "asi" : "asian", "african" : "african", "hispanic":"hispanic people", "mus.wom" : "muslim women"})

In [779]:
c["target"].value_counts().to_dict()

{'none': 15033,
 'women': 2035,
 'black': 1961,
 'jewish': 1096,
 'muslim': 1002,
 'immigrants': 672,
 'refugee': 470,
 'south asians': 338,
 'arabic people': 338,
 'indigenous': 239,
 'mixed people': 201}

In [780]:
#columns to use for DynaHate : text (post), label (offy/n), target (group), (no stereotypes? See how model functions)
#["post", "group", "implications", "whoTarget", "offensiveYN"]
c.drop(["index", "acl.id", "X1", "round", "acl.id.matched", "annotator", "level"], axis=1, index=None, inplace=True)
c.columns=["post", "offensiveYN", "type", "group", "round.base", "split"]
c

,post,offensiveYN,type,group,round.base,split
0,There are too many costas in this country,1,none,none,train,2
1,There is way too many m0ng0l0id kids at my sch...,0,derogation,NaN,train,2
2,I called my brother a twat and my mom got so m...,1,none,none,train,2
3,the worst of them are the taaaaaaaaaards becau...,0,derogation,NaN,train,2
4,the worst of them are the viruses because they...,1,none,none,train,2
...,...,...,...,...,...,...
26868,If she isn't the one for you she's just the on...,1,none,none,dev,4
26869,I have that opinion. absolutely everyone I act...,1,none,none,dev,4
26870,I guess you are assuming that the french laws ...,0,derogation,immigrants,dev,4
26871,"I mean, theres no one better right now to give...",1,none,none,dev,4


In [781]:
#split data into dev, test, and train
dev_dyna, test_dyna, train_dyna = [y for x, y in c.groupby('split', as_index=False)]
dev_dyna.drop(["split"], axis=1, inplace=True)
test_dyna.drop(["split"], axis=1, inplace=True)
train_dyna.drop(["split"], axis=1, inplace=True)
# QUESTION: What rows are necessary, and how are they translatable to SBIC?
# if it's too complicated, you can take only the stage3 posts and see how our model does on the posts that we know have implications? 
# And we can measure the offensiveness classification (ie. stage 1) separately

In [782]:
all_data = pd.DataFrame()
all_data = all_data.append(train_dyna)[train_dyna.columns.tolist()]
all_data = all_data.append(training_csv).dropna(axis=1, how="all")
all_data["whoTarget"] = [1.0] * len(all_data)

In [783]:
all_data["implications"] = all_data["implications"].fillna("[]")

In [784]:
#columns to use for DynaHate : text (post), label (offy/n), target (group), (no stereotypes? See how model functions)
#["post", "group", "implications", "whoTarget", "offensiveYN"]

In [785]:
#merge each file
from functools import reduce
df1 = pd.read_csv("implicit-hate-corpus/implicit_hate_v1_stg1.tsv", sep='\t')
df2 = pd.read_csv("implicit-hate-corpus/implicit_hate_v1_SAP_posts.tsv", sep="\t")
df3 = pd.read_csv("implicit-hate-corpus/implicit_hate_v1_stg2.tsv", sep="\t")
df4 = pd.read_csv("implicit-hate-corpus/implicit_hate_v1_stg3.tsv", sep="\t")
df5 = pd.read_csv("implicit-hate-corpus/implicit_hate_v1_stg2_posts.tsv", sep="\t")
df6 = pd.read_csv("implicit-hate-corpus/implicit_hate_v1_stg3_posts.tsv", sep="\t")
df7 = pd.read_csv("implicit-hate-corpus/implicit_hate_v1_stg1_posts.tsv", sep="\t")

# combine each stage together
stg1_df = df1.merge(df7["post"], left_index=True, right_index=True)
#stg2_df = df3.merge(df5["post"], left_index=True, right_index=True)
stg3_df = df4.merge(df6["post"], left_index=True, right_index=True)
total_df = pd.concat([stg1_df, stg2_df, stg3_df], ignore_index=True)
total_df
#data_frames = [df1, df2, df3, df4, df5, df6, df7]
#df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['ID'],
#                                            how='outer'), data_frames)

,ID,class,post,implicit_class,extra_implicit_class,target,implied_statement
0,399886440588247041,implicit_hate,""" : jewish harvard professor noel ignatiev w...",NaN,NaN,NaN,NaN
1,929901925100937216,not_hate,b.higher education is a part of european cult...,NaN,NaN,NaN,NaN
2,728678509497954304,not_hate,"has a problem with "" the whites "" "" and "" "" ...",NaN,NaN,NaN,NaN
3,625688620444180481,not_hate,is yasir qadhi a hate preacher for calling ch...,NaN,NaN,NaN,NaN
4,441089979322597376,not_hate,"rt "" : how three million germans mass murder...",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
34180,884073715490861056,NaN,your people came to a friendly and highly popu...,NaN,NaN,Asian folks,Asian folks have destroyed the country
34181,603471575833137153,NaN,it's as if sexual abuse follows perkins around .,NaN,NaN,Immigrants,Immigrants are sexual offenders
34182,669074062862368768,NaN,every india lover must boycott these pigs movies,NaN,NaN,Indian folks,Indian folks are pigs
34183,466220911675179008,NaN,ask more rss intellectuals to come on powerful...,NaN,NaN,Indian folks,Indian folks are uneducated


In [786]:
stg1_df.drop(["ID", "class"], axis=1, inplace=True)
stg1_df["offensiveYN"] = ["[OffN]"] * len(stg1_df)
stg1_df["whoTarget"] = [1.0] * len(stg1_df)

In [787]:
#Add offn from stage one to balance out 
# deduplicate non offensive posts (compared to SBIC) 
stg3_df.drop(["ID"], axis=1, inplace=True)
stg3_df.columns = ["group", "implications", "post"]
stg3_df["offensiveYN"] = ["[OffY]"] * len(stg3_df)
stg3_df["whoTarget"] = [1.0] * len(stg3_df)

In [788]:
stg3_df = stg3_df.append(stg1_df)

In [792]:
all_data.append(stg3_df).reset_index()

,index,post,offensiveYN,type,group,round.base,implications,whoTarget
0,17850,Very much so. The refugee migrants are a can o...,0,derogation,refugee,train,[],1.0
1,17851,"Sorry lads, this isn't a happy ever after and ...",0,derogation,refugee,train,[],1.0
2,17852,I feel very happy to live in Eastern Europe ri...,1,none,none,train,[],1.0
3,17853,We offer protection and welcome refugees from ...,0,derogation,refugee,train,[],1.0
4,17854,We offer protection and welcome refugees from ...,1,none,none,train,[],1.0
...,...,...,...,...,...,...,...,...
72281,21475,"Seeing prostitutes is morally wrong, but being...",[OffN],NaN,NaN,NaN,NaN,1.0
72282,21476,I wonder how many females I raped today I spen...,[OffN],NaN,NaN,NaN,NaN,1.0
72283,21477,Having a criminal record is more attractive to...,[OffN],NaN,NaN,NaN,NaN,1.0
72284,21478,Another advantage of being a female: getting b...,[OffN],NaN,NaN,NaN,NaN,1.0


# Accuracy : Structure

- accuracy of structure (are the tokens in the right order/right place? Good first indicator for models to see if at least the structure is learned correctly

#### for scramble model/regular model
- traverse strings - if next token is not is expected:
    - check if it is an incorrect prediction in the first place (those will not count)
    - if it is not an incorrect prediction, check the count on each token. (we can start with a simple true/false, but: structure is more than just if its in the right order and place, it's about if it makes sense)

In [501]:
# for script, make this neat

In [532]:
# merge dataframes
preds = pd.read_csv("predictions_2_1000.csv", index_col=0)
preds.columns = ["input", "generated"]

In [533]:
real_val = pd.read_csv("SBIC.dev.1.csv")

In [534]:
merged_df = real_val.merge(preds, on="input", how="inner")

In [535]:
list_of_data_preds = []
list_of_data_real = []
merged_df["output"] = merged_df["output"].apply(lambda x: x.replace("[boi] ", "")).apply(
    lambda x: x.replace(" [eoi]", ""))

merged_df["generated"] = merged_df["generated"].apply(lambda x: x.replace("<pad> ", "")).apply(
    lambda x: x.replace(" <pad>", ""))

y_data_real = merged_df["output"].values
y_data_preds = merged_df["generated"].values

# filter out incorrect results
for i in range(0, len(merged_df)):
    list_of_data_preds.append(list(filter(lambda x: x != "", [sentence.strip() for sentence in
                                                y_data_preds[i].replace("<pad>", '').replace("[", '').replace("boo",'')
                                                .replace("grp", '').replace("ste", '').replace("eoo", '')
                                                .split(']')])))
    list_of_data_real.append(list(filter(lambda x: x != "", [sentence.strip() for sentence in
                                                y_data_real[i].replace("<pad>", '').replace("[", '').replace("boo",'')
                                                .replace("grp", '').replace("ste", '').replace("eoo", '')
                                                .split(']')])))

list_of_data_real = pd.DataFrame(list_of_data_real)
list_of_data_preds = pd.DataFrame(list_of_data_preds)
list_of_data_real[0] = list_of_data_real[0].apply(lambda label: 1 if label == "OffN" else 0)
list_of_data_preds[0] = list_of_data_preds[0].apply(lambda label: 1 if label == "OffN" else 0)
mask = (list_of_data_real[0] == list_of_data_preds[0])

In [576]:
# merge dataframes
preds = pd.read_csv("predictions_3_pt3.csv", index_col=0)
preds.columns = ["input", "generated"]

In [584]:
preds

,input,generated
0,"[boi] [OffY] [grp] Africans,black folks [ste] ...",<pad> [boo] [bos] What do you call a black kid...
1,"[boi] [OffY] [grp] Africans,black folks [eoi]",<pad> [boo] [bos] What do you call a black per...
2,[boi] [OffY] [eoi],<pad> [boo] [bos]What do you call a black pers...
3,[boi] [bos] You can't tell me that african chi...,"<pad> [boo] [boo] [boo] Africans,black folks,b..."
4,"[boi] [OffY] [grp] black folks,folks with phys...",<pad> [boo] [bos] What do you call a black per...
...,...,...
995,[boi] [OffY] [grp] black folks [eoi],<pad> [boo] [bos] What do you call a black per...
996,[boi] [OffY] [eoi],<pad> [boo] [bos]What do you call a black pers...
997,"[boi] [bos] Wait, is it illegal to keep Blacks...",<pad> [boo] [boo] [boo] black folks [ste] blac...
998,"[boi] [OffY] [grp] conservatives,liberals [ste...",<pad> [boo] [bos] The Left has a slur against ...


In [585]:
real_val

,input,output
0,"[boi] [OffY] [grp] Africans,black folks [ste] ...",[boo] [bos] You can't tell me that african chi...
1,"[boi] [OffY] [grp] Africans,black folks [eoi]",[boo] [bos] You can't tell me that african chi...
2,[boi] [OffY] [eoi],[boo] [bos]You can't tell me that african chil...
3,[boi] [bos] You can't tell me that african chi...,"[boo] [OffY] [grp] Africans,black folks [ste] ..."
4,"[boi] [OffY] [grp] black folks,folks with phys...",[boo] [bos] Why do you never see a black perso...
...,...,...
9023,[boi] [OffY] [eoi],[boo] [bos]What's the difference between an ov...
9024,[boi] [bos] What's the difference between an o...,"[boo] [OffY] [grp] assault victims,kids [ste] ..."
9025,[boi] [bos] Cause it's from 7 years ago? Fucki...,[boo] [OffN] [eoo]
9026,"[boi] [bos] I'm so happy I met you, I couldn't...",[boo] [OffN] [eoo]


In [577]:
real_val = pd.read_csv("SBIC.dev.scramble.3.5050.csv")

In [589]:
#merged_df = real_val.merge(preds, on="input", how="inner")
merged_df = real_val.merge(preds.drop_duplicates(subset=['input']), how='inner')

In [590]:
merged_df

,input,output,generated
0,"[boi] [OffY] [grp] Africans,black folks [ste] ...",[boo] [bos] You can't tell me that african chi...,<pad> [boo] [bos] What do you call a black kid...
1,"[boi] [OffY] [grp] Africans,black folks [eoi]",[boo] [bos] You can't tell me that african chi...,<pad> [boo] [bos] What do you call a black per...
2,"[boi] [OffY] [grp] Africans,black folks [eoi]",[boo] [bos] What do you call a garbage truck i...,<pad> [boo] [bos] What do you call a black per...
3,[boi] [OffY] [eoi],[boo] [bos]You can't tell me that african chil...,<pad> [boo] [bos]What do you call a black pers...
4,[boi] [OffY] [eoi],[boo] [bos]Why do you never see a black person...,<pad> [boo] [bos]What do you call a black pers...
...,...,...,...
3837,[boi] [OffY] [grp] black folks [ste] black fol...,[boo] [bos] During slavery we used to have to ...,<pad> [boo] [bos] What do you call a black per...
3838,[boi] [OffY] [grp] black folks [ste] black fol...,[boo] [bos] What's the difference between a ma...,<pad> [boo] [bos] What do you call a black per...
3839,"[boi] [bos] Wait, is it illegal to keep Blacks...",[boo] [OffY] [grp] black folks [ste] black fol...,<pad> [boo] [boo] [boo] black folks [ste] blac...
3840,"[boi] [OffY] [grp] conservatives,liberals [ste...",[boo] [bos] the whole me-being-republican thin...,<pad> [boo] [bos] The Left has a slur against ...


In [591]:
list_of_data_preds = []
list_of_data_real = []
merged_df["output"] = merged_df["output"].apply(lambda x: x.replace("[boi] ", "")).apply(
    lambda x: x.replace(" [eoi]", ""))

merged_df["generated"] = merged_df["generated"].apply(lambda x: x.replace("<pad> ", "")).apply(
    lambda x: x.replace(" <pad>", ""))

y_data_real = merged_df["output"].values
y_data_preds = merged_df["generated"].values

# filter out incorrect results
for i in range(0, len(merged_df)):
    list_of_data_preds.append(list(filter(lambda x: x != "", [sentence.strip() for sentence in
                                                y_data_preds[i].replace("<pad>", '').replace("[", '').replace("boo",'')
                                                .replace("grp", '').replace("ste", '').replace("eoo", '')
                                                .split(']')])))
    list_of_data_real.append(list(filter(lambda x: x != "", [sentence.strip() for sentence in
                                                y_data_real[i].replace("<pad>", '').replace("[", '').replace("boo",'')
                                                .replace("grp", '').replace("ste", '').replace("eoo", '')
                                                .split(']')])))

list_of_data_real = pd.DataFrame(list_of_data_real)
list_of_data_preds = pd.DataFrame(list_of_data_preds)
list_of_data_real[0] = list_of_data_real[0].apply(lambda label: 1 if label == "OffN" else 0)
list_of_data_preds[0] = list_of_data_preds[0].apply(lambda label: 1 if label == "OffN" else 0)
mask = (list_of_data_real[0] == list_of_data_preds[0])

In [592]:
merged_df_preds = merged_df[np.array(mask,dtype=bool)].reset_index()

In [593]:
merged_df_preds

,index,input,output,generated
0,0,"[boi] [OffY] [grp] Africans,black folks [ste] ...",[boo] [bos] You can't tell me that african chi...,[boo] [bos] What do you call a black kid in a ...
1,1,"[boi] [OffY] [grp] Africans,black folks [eoi]",[boo] [bos] You can't tell me that african chi...,[boo] [bos] What do you call a black person in...
2,2,"[boi] [OffY] [grp] Africans,black folks [eoi]",[boo] [bos] What do you call a garbage truck i...,[boo] [bos] What do you call a black person in...
3,3,[boi] [OffY] [eoi],[boo] [bos]You can't tell me that african chil...,[boo] [bos]What do you call a black person wit...
4,4,[boi] [OffY] [eoi],[boo] [bos]Why do you never see a black person...,[boo] [bos]What do you call a black person wit...
...,...,...,...,...
3667,3837,[boi] [OffY] [grp] black folks [ste] black fol...,[boo] [bos] During slavery we used to have to ...,[boo] [bos] What do you call a black person wi...
3668,3838,[boi] [OffY] [grp] black folks [ste] black fol...,[boo] [bos] What's the difference between a ma...,[boo] [bos] What do you call a black person wi...
3669,3839,"[boi] [bos] Wait, is it illegal to keep Blacks...",[boo] [OffY] [grp] black folks [ste] black fol...,[boo] [boo] [boo] black folks [ste] black folk...
3670,3840,"[boi] [OffY] [grp] conservatives,liberals [ste...",[boo] [bos] the whole me-being-republican thin...,[boo] [bos] The Left has a slur against the Le...


In [537]:
merged_df_regular = merged_df[np.array(mask,dtype=bool)].reset_index()

In [542]:
merged_df_regular

,index,input,output,generated
0,0,[boi] \nRed Nation Rising\n‏\n\n \n@RedNation...,[boo] [OffN] [eoo],[boo] [OffN] [eoo]
1,1,"[boi] they found marijuana, butane hash oil (...",[boo] [OffN] [eoo],[boo] [OffN] [eoo]
2,3,"[boi] """""" animal friend in My ass"""" You need s...",[boo] [OffN] [eoo],[boo] [OffN] [eoo]
3,4,"[boi] """"""A mans belly shall be satisfied with ...",[boo] [OffN] [eoo],[boo] [OffN] [eoo]
4,5,"[boi] """"""Alexa order stain remover"""" """"Go fuck...",[boo] [OffN] [eoo],[boo] [OffN] [eoo]
...,...,...,...,...
797,995,"[boi] Guy next to me on plane:: So, where are ...",[boo] [OffY] [grp] asian folks [ste] asians al...,[boo] [OffY] [grp] asian folks [ste] asian peo...
798,996,"[boi] Guy wants to fuck a girl, not learn basi...",[boo] [OffN] [eoo],[boo] [OffN] [eoo]
799,997,[boi] Guys! I am not Justin Bieber! Stop sendi...,[boo] [OffN] [eoo],[boo] [OffN] [eoo]
800,998,[boi] HAHA @ThaRodickk Im not sexist....but I ...,"[boo] [OffY] [grp] men,women [ste] men make go...",[boo] [OffY] [grp] women [ste] women are bad a...


In [594]:
import re
import numpy as np
total_values = []
# TODO: add in a check to make sure the prediction is correct before adding it to total_values
for i in range(0, len(merged_df_preds)):
    # find all tokens between square brackets
    tokens_input = re.findall("(?<=\[)[^]]+(?=\])", merged_df_preds["output"].tolist()[i])
    tokens_generated = re.findall("(?<=\[)[^]]+(?=\])", merged_df_preds["generated"].tolist()[i])
    #compare the arrays elementwise to determine number of tokens correctly preserved, divide by total amount of tokens preserved
    try:
        total_values.append((np.array(tokens_input)==np.array(tokens_generated)).sum()/len(tokens_input))
    #short term solution to deal with arrays that are not the same length: talk in meeting about best way to mitigate
    except:
        total_values.append(0);
print(np.mean(total_values))

0.6857752360203342


In [115]:
accuracy_structure = np.mean(total_values)

In [74]:
real_val["input"].tolist()[0]

'[boi] [OffY] [grp] Africans,black folks [ste] african kids are starving. [eoi]'

# Regular model

In [391]:
# merge dataframes
preds = pd.read_csv("predictions_2_1000.csv", index_col=0)
preds.columns = ["input", "generated"]

In [392]:
real_val = pd.read_csv("SBIC.dev.1.csv")

In [262]:
merged_df = real_val.merge(preds, on="input", how="inner")

In [263]:
import re
import numpy as np
total_values = []
# TODO: add in a check to make sure the prediction is correct before adding it to total_values
for i in range(0, len(merged_df)):
    # find all tokens between square brackets
    tokens_input = re.findall("(?<=\[)[^]]+(?=\])", merged_df["output"].tolist()[i])
    tokens_generated = re.findall("(?<=\[)[^]]+(?=\])", merged_df["generated"].tolist()[i])
    #compare the arrays elementwise to determine number of tokens correctly preserved, divide by total amount of tokens preserved
    try:
        total_values.append((np.array(tokens_input)==np.array(tokens_generated)).sum()/len(tokens_input))
    #short term solution to deal with arrays that are not the same length: talk in meeting about best way to mitigate
    except:
        total_values.append(0);

print(np.mean(total_values))

0.802


In [115]:
accuracy_structure = np.mean(total_values)

In [74]:
real_val["input"].tolist()[0]

'[boi] [OffY] [grp] Africans,black folks [ste] african kids are starving. [eoi]'

# Accuracy : Generation of Similar values

- frequency of generation (check how frequently the model learns a pattern, generate a heatmap of sorts with keywords)

In [603]:
len(set(merged_df_preds["generated"].tolist()))/len(set(merged_df_preds["output"]))

0.08446866485013624

In [604]:
import nltk
nltk.download('punkt')
from nltk.util import ngrams
 
# Function to generate n-grams from sentences.
def extract_ngrams(data, num):
    n_grams = ngrams(nltk.word_tokenize(data), num)
    return [ ' '.join(grams) for grams in n_grams]

[nltk_data] Downloading package punkt to /home/nandsra/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [609]:
gen_vs_act = []
gen_vs_gen = []
for i in range(0, len(merged_df_preds)):
    generated = merged_df_preds["generated"].tolist()[i].replace("<pad>", "").replace("[eoo]", "").replace("[cls]", "").replace("[boo]","")
    output = merged_df_preds["output"].tolist()[i].replace("<pad>", "").replace("[eoo]", "").replace("[cls]", "").replace("[boo]","")
    if ("[bos]" in generated) and ("[bos]" in output):
        reference = generated.split("[bos]")[1].strip()
        candidate = output.split("[bos]")[1].strip()
        gen = extract_ngrams(reference, 4)
        act = extract_ngrams(candidate, 4)
        # generated vs actual
        gen_vs_act.append(len(set(gen))/len(set(act)))
        # generated unique vs generated total
        gen_vs_gen.append(len(set(gen))/len(gen))

In [610]:
np.mean(gen_vs_act)

0.838394274707409

# Accuracy : BLEU scores for the implication score

In [214]:
!pip install nltk

     |████████████████████████████████| 1.5 MB 7.1 MB/s eta 0:00:01


In [497]:
from nltk.translate.bleu_score import sentence_bleu

In [612]:
# get data within the implication bounds 
list_preds = merged_df_preds["generated"].tolist()
import re
total_val = []
#split_val = [preds.split("[ste]") for preds in list_preds]
for i in range(0, len(list_preds)):
    list_preds[i] = list_preds[i].replace("<pad>", "").replace("[eoo]", "").replace("[cls]", "").replace("[boo]","")
    real_val_list[i] = real_val_list[i].replace("<pad>", "").replace("[eoo]", "").replace("[cls]", "").replace("[boo]","")
    if "[ste]" in list_preds[i] and "[ste]" in real_val_list[i]:
        reference = [list_preds[i].split("[ste]")[1].strip().split(" ")]
        candidate = real_val_list[i].split("[ste]")[1].strip().split(" ")
        twogram = sentence_bleu(reference, candidate, weights=(0.5, 0.5, 0, 0), smoothing_function=SmoothingFunction(epsilon=1e-12).method1)
        threegram = sentence_bleu(reference, candidate, weights=(0.33, 0.33, 0.33, 0), smoothing_function=SmoothingFunction(epsilon=1e-12).method1)
        total_val.append((twogram + threegram) / 2)
# 2 and 3 gram mean for all the data 
# compare data to actual implications using BLEU score
# smoothing method1 - NLTK sentence_bleu add
# steps: find where the data that takes implications is in the actual data, compare it with the other one

In [613]:
np.mean(total_val)

0.09106520124850005

In [474]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

In [614]:
# get data within the implication bounds 
list_preds = merged_df_regular["generated"].tolist()
import re
total_val = []
#split_val = [preds.split("[ste]") for preds in list_preds]
for i in range(0, len(list_preds)):
    list_preds[i] = list_preds[i].replace("<pad>", "")
    if "[ste]" in list_preds[i] and "[ste]" in real_val_list[i]:
        reference = [list_preds[i].split("[ste]")[1].strip().split(" ")]
        candidate = real_val_list[i].split("[ste]")[1].strip().split(" ")
        twogram = sentence_bleu(reference, candidate, weights=(0.5, 0.5, 0, 0), smoothing_function=SmoothingFunction(epsilon=1e-12).method1)
        threegram = sentence_bleu(reference, candidate, weights=(0.33, 0.33, 0.33, 0), smoothing_function=SmoothingFunction(epsilon=1e-12).method1)
        total_val.append((twogram + threegram) / 2)
# 2 and 3 gram mean for all the data 
# compare data to actual implications using BLEU score
# smoothing method1 - NLTK sentence_bleu add
# steps: find where the data that takes implications is in the actual data, compare it with the other one

In [615]:
np.mean(total_val)

0.00929329002353653

# Accuracy of 50/50 model

In [87]:
preds = pd.read_csv("predictions_2_1000.csv", index_col=0)

In [88]:
preds["Source Text"] = preds["Source Text"].apply(lambda x: x.replace("[boi] ", "")).apply(lambda x: x.replace(" [eoi]", ""))

In [90]:
preds["Generated Text"] = preds["Generated Text"].apply(lambda x: x.replace("<pad> ", "")).apply(lambda x: x.replace(" <pad>", ""))
preds.columns = ["post", "output"]

In [28]:
from sklearn.metrics import f1_score
import re
list_of_data = []
# split on parenthesis
for i in range(0, len(generated_predictions)):
    list_of_data.append(list(filter(lambda x: x != "", [sentence.strip() for sentence in
                                                generated_predictions[i].replace("<pad>", '').replace("[", '').replace("boo",'')
                                                .replace("grp", '').replace("ste", '').replace("eoo", '')
                                                .split(']')])))

In [91]:
y_test = pd.read_csv("SBIC.v2.agg.dev.csv", index_col=0)[:1000]
y_test

,post,targetMinority,targetCategory,targetStereotype,whoTarget,intentYN,sexYN,offensiveYN,dataSource,hasBiasedImplication
0,\nRed Nation Rising\n‏\n\n \n@RedNationRising...,[],[],[],NaN,0.110000,0.000000,0.000000,Gab,1
1,"they found marijuana, butane hash oil (BHO) a...",[],[],[],0.0,0.110000,0.000000,0.166667,Gab,1
2,""""""" I have no problem being called Benny! Just...",[],[],[],0.0,0.553333,0.000000,0.500000,t/founta,1
3,""""""" animal friend in My ass"""" You need some He...",[],[],[],0.0,0.886667,0.166667,1.000000,t/founta,1
4,"""""""A mans belly shall be satisfied with the fr...",[],[],[],0.0,0.000000,0.000000,0.000000,t/founta,1
...,...,...,...,...,...,...,...,...,...,...
996,"Guy next to me on plane:: So, where are you fr...","[""asian folks""]","[""race""]","[""asians all look alike"", ""asians are quiet"", ...",1.0,0.830000,0.000000,1.000000,redditMicroagressions,0
997,"Guy wants to fuck a girl, not learn basic carp...",[],[],[],0.0,0.330000,1.000000,0.500000,redditMicroagressions,1
998,Guys! I am not Justin Bieber! Stop sending me ...,[],[],[],NaN,0.000000,0.000000,0.000000,t/founta,1
999,HAHA @ThaRodickk Im not sexist....but I honest...,"[""men"", ""women""]","[""gender""]","[""men make good broadcasters"", ""men only care ...",1.0,1.000000,0.000000,1.000000,t/waseem,0


In [92]:
merged_df = pd.merge(preds, y_test, on="post")
merged_df["offensiveYN"] = merged_df["offensiveYN"].apply(lambda label: 1 if label == 1 else 0)

In [93]:
merged_df["offYNPredictions"] = merged_df["output"].apply(lambda label: 1 if "OffY" in label else 0)

In [110]:
subset_merged_df = merged_df[["output", "targetMinority", "targetCategory", "targetStereotype", "offensiveYN"]]
subset_merged_df[merged_df.output.str.contains('OffY')].head(50)

,output,targetMinority,targetCategory,targetStereotype,offensiveYN
2,[boo] [OffY] [grp] women [ste] women are bitches [eoo],[],[],[],0
19,[boo] [OffY] [grp] mentally disabled folks [ste] mentally disabled folks are retarded [eoo],"[""folks with mental illness/disorder, mentally disabled folks"", ""mentally disabled folks""]","[""disabled""]","[""being retarded is bad"", ""makes light of the word retarded"", ""mentally disabled folks are retards."", ""mentally disabled folks should have their disability used as a punchline.""]",1
27,[boo] [OffY] [grp] homeless [ste] homeless people are worthless [eoo],[],[],[],0
41,[boo] [OffY] [grp] liberals [ste] liberals are stupid [eoo],"[""liberals""]","[""social""]","[""are bad"", ""are corrupt""]",0
43,"[boo] [OffY] [grp] homos,gay men [ste] homos are a derogatory term [eoo]","[""gay men""]","[""gender""]","[""calls the homo"", ""he accuses of sexual assault""]",1
44,[boo] [OffY] [grp] asian folks [ste] asian people are a burden on society [eoo],[],[],[],0
53,[boo] [OffY] [grp] women [ste] women are hoes [eoo],"[""women""]","[""gender""]","[""women are easy to manipulate"", ""women are hoes""]",1
54,[boo] [OffY] [grp] women [ste] women are bitches [eoo],[],[],[],0
57,[boo] [OffY] [grp] women [ste] women are bitches [eoo],"[""women""]","[""gender""]","[""women are gold diggers"", ""women are interested in a man's money"", ""women are only after men's money"", ""women only care about material objects""]",1
61,[boo] [OffY] [grp] black folks [ste] black people are niggers [eoo],"[""black folks""]","[""race""]","[""black folks are all very dark skinned"", ""black folks are n*ggers"", ""black folks are niggers"", ""black folks can be mocked"", ""blacks are called the n word"", ""post degrates black folks""]",1


In [95]:
sum_int = sum(merged_df["offYNPredictions"] == merged_df["offensiveYN"])

accuracy_score = sum_int / 1000

f1_val = f1_score(y_test_off, list_of_data[0])


In [96]:
accuracy_score

0.75

In [ ]:
    
    # Qualitative Testing

    match = []
    for i in range(0, len(list_of_data)):
        if list(y_test["offensiveYN"].values)[i] == 0:
            list_of_data_val = str(list(list_of_data[1].values)[i])
            y_test_val = list(y_test["group"].values)
            match.append(any(list_of_data_val in s for s in y_test_val))
            
    raw_accuracy_group = sum(match) / len(match)


    with open(args.output, 'w') as f:
        f.write("raw accuracy of the Offensive YN: \n" + str(accuracy_score))
        f.write("f1 accuracy score of Offensive YN: \n" + str(f1_val))
        f.write("raw accuracy score of the group tokens: \n" + str(raw_accuracy_group))

    f.close()